In [4]:
import flickrapi
import requests
import os
from urllib.parse import urlparse

# Your Flickr API key and secret
api_key = '081bf0c4fa3437478b3bcfb747d418e4'
api_secret = '1b7274b2f4129a69'

# Initialize the Flickr API client
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')


In [ ]:
# Directories to save images
indoor_dir = 'flickr_indoor_images'
outdoor_dir = 'flickr_outdoor_images'
os.makedirs(indoor_dir, exist_ok=True)
os.makedirs(outdoor_dir, exist_ok=True)

# Define search queries for indoor and outdoor scenes
indoor_queries = ['living room', 'office', 'bedroom', 'kitchen', 'restaurant interior', 'classroom']
outdoor_queries = ['mountain landscape', 'park', 'city street', 'beach', 'forest trail', 'outdoor market']

# Function to download images from multiple search queries
def download_images_from_queries(queries, directory, num_images):
    count = 0
    for query in queries:
        photos = flickr.photos.search(
            text=query, 
            per_page=num_images // len(queries), 
            media='photos', 
            content_type=1, 
            sort='relevance',
            license='1,2,3,4,5,6,7',  # Optional: filter by Creative Commons licenses
            extras='url_o,url_l,url_c,url_z'  # Fetch various sizes
        )['photos']['photo']
        
        for photo in photos:
            try:
                # Try to download the largest available size
                url = photo.get('url_o') or photo.get('url_l') or photo.get('url_c') or photo.get('url_z')
                if url:
                    response = requests.get(url, stream=True)
                    parsed_url = urlparse(url)
                    filename = os.path.basename(parsed_url.path)
                    image_path = os.path.join(directory, filename)

                    # Save the image
                    with open(image_path, 'wb') as f:
                        f.write(response.content)

                    count += 1
                    print(f"Downloaded {count}/{num_images} images for {query}")

                # Stop if we reach the total number of images needed
                if count >= num_images:
                    break
            except Exception as e:
                print(f"Error downloading image {photo['id']}: {e}")
        if count >= num_images:
            break

# Download indoor and outdoor images using specific scene search terms
download_images_from_queries(indoor_queries, indoor_dir, 500)
download_images_from_queries(outdoor_queries, outdoor_dir, 500)
